In [3]:
import numpy as np
import pandas as pd
import pickle

In [12]:
# Load the dataset
df = pd.read_csv("casos_covid19_2020.csv")  # Adjust delimiter if needed

# Convert column names to lowercase for consistency
df.columns = df.columns.str.lower()

# Keep only rows where "provincia" is CABA
df_caba = df[df["provincia"] == "CABA"]

# Convert 'fecha_clasificacion' to datetime format
df_caba["fecha_clasificacion"] = pd.to_datetime(df_caba["fecha_clasificacion"], errors="coerce", format="%d%b%Y")

# Drop any rows with invalid or missing classification dates
df_caba = df_caba.dropna(subset=["fecha_clasificacion"])

# Ensure 'comuna' column exists and is not null
if "comuna" in df_caba.columns:
    df_caba = df_caba.dropna(subset=["comuna"])

    # Count cases per day and comuna
    daily_cases = df_caba.groupby(["fecha_clasificacion", "comuna"]).size().reset_index(name="daily_cases")

    # Sort by date and comuna
    daily_cases = daily_cases.sort_values(by=["comuna", "fecha_clasificacion"])

    # Compute cumulative cases per comuna
    daily_cases["cumulative_cases"] = daily_cases.groupby("comuna")["daily_cases"].cumsum()

    # Display result
    print(daily_cases.head())  # Adjust for full display
else:
    print("Column 'comuna' not found in the dataset.")

/tmp/ipykernel_1902306/3250208569.py:2: DtypeWarning: Columns (10,11,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("casos_covid19_2020.csv")  # Adjust delimiter if needed


   fecha_clasificacion  comuna  daily_cases  cumulative_cases
17          2020-03-05     1.0            2                 2
29          2020-03-11     1.0            1                 3
35          2020-03-12     1.0            2                 5
42          2020-03-13     1.0            2                 7
50          2020-03-15     1.0            3                10


/tmp/ipykernel_1902306/3250208569.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_caba["fecha_clasificacion"] = pd.to_datetime(df_caba["fecha_clasificacion"], errors="coerce", format="%d%b%Y")


In [22]:
# Assuming daily_cases is your DataFrame
daily_cases = daily_cases.rename(columns={'fecha_clasificacion': 'date'})
daily_cases = daily_cases.rename(columns={'cumulative_cases': 'total_cases'})
daily_cases

,date,comuna,daily_cases,total_cases
17,2020-03-05,1.0,2,2
29,2020-03-11,1.0,1,3
35,2020-03-12,1.0,2,5
42,2020-03-13,1.0,2,7
50,2020-03-15,1.0,3,10
...,...,...,...,...
5959,2022-01-15,15.0,1,14910
5963,2022-01-16,15.0,2,14912
5972,2022-01-17,15.0,1,14913
5983,2022-01-18,15.0,1,14914


In [23]:
with open("BA_Covid_Data.pkl", "wb") as f:
    pickle.dump(daily_cases, f)